#### HERE API

- Saving driving times using the api, for the 3 vector types 

In [1]:
import pandas as pd
import plotly.express as px
import flexpolyline as fp
import requests
from datetime import datetime, timedelta

In [2]:
import time

In [3]:
here_api_key = "pqtZHXxlRn6W4Ej-Or_m9J49HFBpQRk1F4hUSXxrH8s"

In [4]:
departure_time_iso = '2021-02-22T10:00:00'

In [5]:
isochrone_url = (
f'https://isoline.router.hereapi.com/v8/isolines?transportMode=car&origin={51.3136784839102},{4.941369315629484}&range[type]=time&range[values]={60*30}&departureTime={departure_time_iso}&apikey={here_api_key}'
)

In [6]:
# aed_latlons  = pd.read_parquet(r"D:\Rahul\Stats and Data Science\Sem 4\Datathon 24\aed_locations_latlon.parquet.gzip")
pit_latlons = pd.read_parquet("pit_locations_latlon.parquet.gzip")
mug_latlons = pd.read_parquet("mug_locations_latlon.parquet.gzip")

In [7]:
amb_latlons = pd.read_parquet('ambulance_locations.parquet.gzip')

In [8]:
amb_latlons

,base,medical_resource,province,region,departure_location,departure_location_number,latitude,longitude,occasional_permanence
0,BA ANTW [Borgerhout] AMBUCE,ZW ANTWERPEN 7 (Team: AAANTW07A),Antwerpen,Vlaams Gewest,Gijselstraat 150 2140 BORGERHOUT,3811,51.215516,4.443788,N
1,BA ANTW [Hoboken] AMBUCE,ZW ANTWERPEN 9 (Team: AAANTW09A),Antwerpen,Vlaams Gewest,Antwerpsesteenweg 242 2660 HOBOKEN,3812,51.178831,4.361948,N
2,BA ANTW [Hof Ter Schelde] AC Antwerpen,ZW ANTWERPEN 3 (Team: AAANTW03A),Antwerpen,Vlaams Gewest,August Vermeylenlaan 6 2050 ANTWERPEN,10902,51.232810,4.377237,N
3,BA ANTW [Kaai 606] SIHWA,ZW ANTWERPEN HAVEN 1 (Team: AAAPOR01A)_x000D_Z...,Antwerpen,Vlaams Gewest,Antwerpsebaan 4 (Kaai 606) 2040 ANTWERPEN,12902,51.315516,4.321506,N
4,BA ANTW [Sint Elisabeth] AC Antwerpen,ZW ANTWERPEN 2 (Team: AAANTW02A),Antwerpen,Vlaams Gewest,Leopoldstraat 26 2000 ANTWERPEN,10901,51.213623,4.405718,N
...,...,...,...,...,...,...,...,...,...
274,FX PDS SHUB Luxembourg,ZS.STHUBERT 1 (Team: AXSHUB01A)_x000D_ZS.STHUB...,Luxemburg,Waals Gewest,Route De Poix 23 6870 SAINT-HUBERT,80003,50.025633,5.359641,Y
275,HG UR HERS Andre Renard,AMB OCC HERSTAL (Team: AGTIHARE1),Luik,Waals Gewest,Rue André Renard 1 4040 HERSTAL,63601,50.674803,5.633665,Y
276,HG UR LIEG CHR Citadelle,AMB LIEGE CHR (Team: AGLIEG09A)_x000D_AMB OCC ...,Luik,Waals Gewest,Allée De L'Absent 1 4000 LIEGE,67201,50.652888,5.578356,Y
277,HG UR LIEG CHU Notre Dame Bruyeres,(Team: AGTIHCHU1),Luik,Waals Gewest,Rue de Gaillarmont 600 4030 LIEGE,63602,50.622523,5.636381,Y


In [9]:
amb_latlons.rename(columns={'latitude':'Lat', 'longitude':'Lon'}, inplace=True)

In [10]:
# aed_latlons['DT_15'] = None
pit_latlons['DT_3'] = None
mug_latlons['DT_3'] = None
amb_latlons['DT_3'] = None

In [11]:
# for index, row in aed_latlons.iterrows():
#     # print(index)
#     print(aed_latlons.at[index, 'DT_15'])

In [12]:
# fig = px.scatter_mapbox(
#     temp_polydf,
#     lat='Lat',
#     lon='Lon',
#     zoom=14,  # Adjust the zoom level as needed
#     title='Isoline Polygon Representation'
# )
# fig.add_trace(
#     px.line_mapbox(temp_polydf, lat='Lat', lon='Lon').data[0]
# )
# fig.update_layout(mapbox_style="open-street-map")

In [13]:
def store_coords(df):
    print('\nStart of store coords')
    for index, row in df.iterrows():
        print('\n')
        print('-'*10)
        print(f'\n Loop iteration {index}')
        lat = df.at[index, 'Lat']
        lon = df.at[index, 'Lon']
        isochrone_url = (
                        f'https://isoline.router.hereapi.com/v8/isolines?transportMode=car&origin={lat},{lon}&range[type]=time&range[values]={60*3}&departureTime={departure_time_iso}&apikey={here_api_key}'
                        )
        response = requests.get(isochrone_url)
        df.at[index, 'DT_3'] = fp.decode(response.json()['isolines'][0]['polygons'][0]['outer'])
        print('\nStored polygon in df')
        print('\n Sleeping for 1 sec....')
        time.sleep(1)
        print('\nAwake, starting again...')
    return 1

In [14]:
# demo_isochrone_url = (
#                         f'https://isoline.router.hereapi.com/v8/isolines?transportMode=car&origin={51.215516},{4.443788}&range[type]=time&range[values]={60*6}&departureTime={departure_time_iso}&apikey={here_api_key}'
#                         )

In [15]:
# demo_response = requests.get(demo_isochrone_url)
# demo_response.json()
# temp = pd.DataFrame(fp.decode(demo_response.json()['isolines'][0]['polygons'][0]['outer']), columns=['Lat', 'Lon'])

In [16]:
# fig = px.scatter_mapbox(
#     temp,
#     lat='Lat',
#     lon='Lon',
#     zoom=14,  # Adjust the zoom level as needed
#     title='Isoline Polygon Representation'
# )
# fig.add_trace(
#     px.line_mapbox(temp, lat='Lat', lon='Lon').data[0]
# )
# fig.update_layout(mapbox_style="open-street-map")

In [17]:
pit_cpy = pit_latlons.copy(deep=True)
mug_cpy = mug_latlons.copy(deep=True)
# aed_cpy = aed_latlons.copy(deep=True)
amb_cpy = amb_latlons.copy(deep=True)

In [18]:
store_coords(pit_latlons)


Start of store coords


----------

 Loop iteration 0

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 1

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 2

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 3

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 4

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 5

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 6

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 7

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 8

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 9


1

In [19]:
pit_latlons

,unit,campus,province,region,unit_id,ambucode,ambusitecode,link,Full_Address,Lat,Lon,DT_3
0,"100001: PIT Rumst, AZ Rivierland",Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- ...,Antwerpen/Anvers,Vlaanderen/Flandre,PARUMS01A,191,19101,None,"1270 RUMST,Antwerpen,Belgium",51.101087,4.412720,"[(51.101532, 4.388394), (51.101532, 4.390068),..."
1,"102001: PIT Deinze, St Vincentius",Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS -...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,PODEIN01A,413,41301,None,"1830 AZ SINT-VINCENTIUS DEINZE,Oost-Vlaanderen...",50.986082,3.528019,"[(50.983815, 3.517492), (50.983901, 3.517556),..."
2,102101: PIT Stuivenberg,Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK AN...,Antwerpen/Anvers,Vlaanderen/Flandre,PAANTW01A,117,11704,None,"1180 ZNA STUIVENBERG,Antwerpen,Belgium",51.222321,4.435068,"[(51.223111, 4.424894), (51.22324, 4.424915), ..."
3,"110001: PIT Lier, Heilig Hart",Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PALIER01A,171,17101,None,"1260 HEILIG HART,Antwerpen,Belgium",51.203294,4.418431,"[(51.198177, 4.408457), (51.198177, 4.408607),..."
4,113001: PIT HEILIG HART ZIEKENHUIS,Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PAMOL_01A,190,19001,None,"2080 HEILIG HART ZIEKENHUIS MOL,Antwerpen,Belgium",51.187084,5.114250,"[(51.186204, 5.099008), (51.18629, 5.099072), ..."
5,200001: PIT LOKEREN,Ziekenhuis/Hôpital: 595 - VITAZ --- Campus/Sit...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,POLOKE01A,440,44001,None,"1650 VITAZ CAMPUS LOKEREN,Oost-Vlaanderen,Belgium",51.101206,4.000920,"[(51.107068, 3.989904), (51.107154, 3.989968),..."
6,"210001: PIT ASSE, OLV",Ziekenhuis/Hôpital: 126 - ONZE-LIEVE-VROUW ZIE...,Vlaams Brabant/Brabant flamand,Vlaanderen/Flandre,PVASSE01A,480,48001,None,"1300 OLV ZIEKENHUIS ASSE,Vlaams Brabant,Belgium",50.909713,4.195518,"[(50.897684, 4.175105), (50.897856, 4.175234),..."
7,301001: PIT ST Jan (Associatie St Lucas),Ziekenhuis/Hôpital: 049 - AZ SINT-JAN BRUGGE -...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG01A,391,39102,PIT 301001 gelinkt aan de ziekenhuizen/lié aux...,"1010 AZ SINT-JAN BRUGGE,West-Vlaanderen,Belgium",51.220115,3.193459,"[(51.222253, 3.188589), (51.222382, 3.18861), ..."
8,301001: PIT St Lucas Brugge (associatie St Jan),Ziekenhuis/Hôpital: 140 - AZ SINT-LUCAS --- Ca...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG02A,391,39101,PIT 301001 gelinkt aan de ziekenhuizen/lié aux...,"1000 SINT- LUCAS,West-Vlaanderen,Belgium",51.185125,3.249138,"[(51.187191, 3.246481), (51.187148, 3.246589),..."
9,303001: PIT Blankenberge,Ziekenhuis/Hôpital: 392 - AZ ZENO --- Campus/S...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBLAN01A,374,37401,None,"1040 AZ ZENO BLANKENBERGE,West-Vlaanderen,Belgium",51.308001,3.124703,"[(51.308727, 3.101771), (51.30877, 3.101878), ..."


In [20]:
pit_latlons.to_csv('pit_dt3.csv')

In [21]:
store_coords(mug_latlons)

mug_latlons.to_csv('mug_dt3.csv')


Start of store coords


----------

 Loop iteration 0

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 1

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 2

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 3

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 4

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 5

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 6

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 7

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 8

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 9


In [22]:
store_coords(amb_latlons)

amb_latlons.to_csv('amb_dt3.csv')


Start of store coords


----------

 Loop iteration 0

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 1

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 2

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 3

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 4

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 5

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 6

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 7

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 8

Stored polygon in df

 Sleeping for 1 sec....

Awake, starting again...


----------

 Loop iteration 9
